In [4]:

import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

! pwd
! source /home/qcr/cgras_moveit_ws/devel/setup.bash

/home/qcr/cgras_moveit_ws/src/cgras_robot/src


In [5]:
rospy.init_node("move_group_python_interface_tutorial", anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)
# move_group.set_pose_reference_frame('world')

print(f'pose reference frame: {move_group.get_pose_reference_frame()}')
end_effector_link = move_group.get_end_effector_link()
print(f'end-effector links: {end_effector_link}')

[ INFO] [1703073985.343104907]: Ready to take commands for planning group manipulator.


In [70]:
link_names = robot.get_link_names()
print(link_names)

for link in link_names:
    link_pose = move_group.get_current_pose(link)
    print(f'{link}\n{link_pose.pose.position}')
    print(f'roll, pitch, yaw: {move_group.get_current_rpy(link)}')
    #print(f'{link_pose.header.frame_id}')

['trolley', 'trolley_robot_base', 'base_link', 'base', 'base_link_inertia', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'flange', 'tool0', 'enclosure', 'ee_control_link']
trolley
x: 0.0
y: 0.0
z: 0.0
roll, pitch, yaw: [0.0, -0.0, 0.0]
trolley_robot_base
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, -0.0, 0.0]
base_link
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, -0.0, 1.5707963267948963]
base
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, 0.0, -1.5707963267948968]
base_link_inertia
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, 0.0, -1.5707963267948968]
shoulder_link
x: 0.0
y: 0.0
z: 1.2273
roll, pitch, yaw: [0.0, -0.0, 3.0868036732051034]
upper_arm_link
x: 0.0
y: 0.0
z: 1.2273
roll, pitch, yaw: [-1.5707963270570027, 0.6720926535897931, -0.05478898054788348]
forearm_link
x: -0.4781838977720488
y: 0.026225454999761772
z: 1.6083464347314989
roll, pitch, yaw: [1.5707963270020442, -0.14060000000000003, 3.0868036731760746]
wrist_1_link
x: 0

In [24]:
print('RobotCommander')
# print(f'current state: {robot.get_current_state()}')
print(f'variable values: {robot.get_current_variable_values()}')
print(f'default owner group: {robot.get_default_owner_group("base_link")}')
group_names = robot.get_group_names()
print(f'group names: {group_names}')
for group_name in group_names:
    commander = robot.get_group(group_name)
joint_names = robot.get_joint_names()
print(f'joint names: {joint_names}')
for joint_name in joint_names:
    the_joint = robot.get_joint(joint_name)
print(f'planning frame: {robot.get_planning_frame()}')
print(f'root link: {robot.get_root_link()}')

RobotCommander
variable values: {'elbow_joint': 2.6101, 'shoulder_lift_joint': -2.4695, 'shoulder_pan_joint': 4.6576, 'wrist_1_joint': -1.7818, 'wrist_2_joint': -1.5317, 'wrist_3_joint': 0.5939}
default owner group: None
group names: ['enclosure', 'manipulator']
joint names: ['virutal joint', 'trolley_to_base_joint', 'robot_base_to_robot_joint', 'base_link-base_fixed_joint', 'base_link-base_link_inertia', 'shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'wrist_3-flange', 'flange-tool0', 'enclosure_joint', 'ee_control_joint']
planning frame: trolley
root link: trolley


In [44]:
obj_pose = move_group.get_current_pose(end_effector_link)
obj_pose.pose.position.x -= 0.05
obj_pose.header.frame_id = 'trolley'
move_group.set_pose_target(obj_pose)
success = move_group.go(wait=True)
if success:
    print('Successful Pose Move Goal')
else:
    print('Failed')
move_group.stop()
move_group.clear_pose_targets()
print(f'COMPLETED: {move_group.get_current_pose(end_effector_link).pose}')

Successful Pose Move Goal
COMPLETED: position: 
  x: 0.3792278980395307
  y: 0.05634155796744547
  z: 1.2020534464956403
orientation: 
  x: -0.8944546054567342
  y: -0.44537652893094337
  z: -0.039857471097785674
  w: 0.001445078666313955


In [66]:
move_group.set_named_target('stow')
success = move_group.go(wait=True)
if success:
    print('Successful Pose Move Goal')
else:
    print('Failed')
move_group.stop()
move_group.clear_pose_targets()
print(f'COMPLETED: {move_group.get_current_pose(end_effector_link).pose}')


Successful Pose Move Goal
COMPLETED: position: 
  x: 0.37925071090382156
  y: 0.0564579765250299
  z: 1.201932332000238
orientation: 
  x: 0.8945744278123962
  y: 0.4451000949293221
  z: 0.04027883766560947
  w: 0.00033739344962618835


In [8]:
def display_trajectory(robot, plan):
    global display_trajectory_publisher
    if 'display_trajectory_publisher' not in globals():
        display_trajectory_publisher = rospy.Publisher(
            "/move_group/display_planned_path",
            moveit_msgs.msg.DisplayTrajectory,
            queue_size=20,
        )
    display_trajectory = moveit_msgs.msg.DisplayTrajectory()
    display_trajectory.trajectory_start = robot.get_current_state()
    display_trajectory.trajectory.append(plan)
    display_trajectory_publisher.publish(display_trajectory)

waypoints = []
scale = 1
wpose = move_group.get_current_pose().pose
wpose.position.z += scale * 0.1  # First move up (z)
wpose.position.y += scale * 0.2  # and sideways (y)
waypoints.append(copy.deepcopy(wpose))

wpose.position.x += scale * 0.1  # Second move forward/backwards in (x)
waypoints.append(copy.deepcopy(wpose))

wpose.position.y -= scale * 0.1  # Third move sideways (y)
waypoints.append(copy.deepcopy(wpose))

(plan, fraction) = move_group.compute_cartesian_path(
    waypoints, 0.01, 0.0  # waypoints to follow  # eef_step # jump_threshold
)
print(f'plan: {plan}\nfraction: {fraction}')
display_trajectory(robot, plan)

success = move_group.execute(plan, wait=True)
if success:
    print('Successful Pose Move Goal')
else:
    print('Failed')

plan: joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "trolley"
  joint_names: 
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
  points: 
    - 
      positions: [4.972313968147455, -2.095197586227092, 2.324192106996358, -1.8788208203036532, -1.555387502337821, 0.9078461705649158]
      velocities: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      accelerations: [0.9853146365957467, 0.0, 0.0, 0.0, 0.0, 0.0]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [4.99499976958834, -2.0842382819514693, 2.311468338477119, -1.8773845665296174, -1.557182276827062, 0.9304632487245825]
      velocities: [0.18116155963381023, 0.08960114300520566, -0.10429620200016498, 0.012301563891451144, -0.014371500570002377, 0.1806065569683275]
      accelerations: [1.0025903237176272, 0.512032408597368, -0.5980470961064591, 0.074532

In [11]:
def wait_for_state_update(scene, object_name, box_is_known=False, box_is_attached=False, timeout=4):
        start = rospy.get_time()
        seconds = rospy.get_time()
        while (seconds - start < timeout) and not rospy.is_shutdown():
            # Test if the box is in attached objects
            attached_objects = scene.get_attached_objects([object_name])
            is_attached = len(attached_objects.keys()) > 0
            # Test if the box is in the scene.
            # Note that attaching the box will remove it from known_objects
            is_known = object_name in scene.get_known_object_names()

            # Test if we are in the expected state
            if (box_is_attached == is_attached) and (box_is_known == is_known):
                return True
            # Sleep so that we give other threads time on the processor
            rospy.sleep(0.1)
            seconds = rospy.get_time()

        # If we exited the while loop without returning then we timed out
        return False
        ## END_SUB_TUTORIAL

In [16]:
move_group.attach_object('tank', link_name='trolley')

True

In [14]:
the_tank_obj = moveit_msgs.msg.CollisionObject()
the_tank_obj.id = 'tank_left'
the_tank_obj.header.frame_id = 'trolley'
the_tank_obj.operation = the_tank_obj.ADD
scene.add_object(the_tank_obj)

# col_obj = moveit_msgs.msg.AttachedCollisionObject()
# # col_obj.link_name = 'tank_left'
# col_obj.object.header.frame_id = 'tank_left'
# # col_obj.object.id = "tank"
# # col_obj.object.operation = col_obj.object.ADD
# scene.add_object(the_tank_obj)

wait_for_state_update(scene, "tank", box_is_known=True, timeout=4)

object_names = scene.get_known_object_names(with_type=True)
print(object_names)

object_names = scene.get_objects()
print(object_names)

def publish_attached(self, attached_obj, side):
    attached_obj.operation = CollisionObject.ADD
    msg = AttachedCollisionObject()
    msg.object = attached_obj
    msg.link_name = side+"_gripper" 
    touch_links = [side+"_gripper", side+"_gripper_base", side+"_hand_camera", side+"_hand_range", "octomap"]
    self.attached_pub.publish(msg)

[]
{}
